In [15]:
import pickle
import scipy
import numpy as np
from sklearn import preprocessing, svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler, normalize
from tqdm import tqdm
import pandas as pd

In [4]:
with open('../data/hyperlex_pairs_ppl.pickle', 'rb') as f:
    t = pickle.load(f)

ppl_dict = dict()
for elements in t.items():
    v1 = elements[0][0].split('(')[0].strip(' ')
    v2 = elements[0][1].strip(',')
    ppl = elements[1]
    ppl_dict[(v1, v2)] = ppl

In [143]:
train_df = {'forward': [], 'backward': [],  'label': [], 'from': [], 'to': []}
test_df = {'forward': [], 'backward': [],  'label': []}


with open('../data/hyperlex/splits/lexical/hyperlex_training_all_lexical.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]
        score_10 = splitted[5]
        if (v1, v2) in ppl_dict.keys():
            train_df['forward'].append(ppl_dict[(v1, v2)])
            train_df['backward'].append(ppl_dict[(v2, v1)])
            train_df['label'].append(float(score_10))
            train_df['from'].append(v2)
            train_df['to'].append(v1)
        else:
            lost_train += 1

with open('../data/hyperlex/splits/lexical/hyperlex_test_all_lexical.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]
        score_10 = splitted[5]
        if (v1, v2) in ppl_dict.keys():
            test_df['forward'].append(ppl_dict[(v1, v2)])
            test_df['backward'].append(ppl_dict[(v2, v1)])
            test_df['label'].append(float(score_10))
        else:
            lost_test += 1

In [144]:
train = pd.DataFrame(train_df)
test = pd.DataFrame(test_df)

In [195]:
y = np.array(test['label'])

norm_test_pred = normalize([np.array(test['forward'])], norm='l2')[0].reshape(-1, 1).flatten()
print('Spearman: ', scipy.stats.spearmanr(norm_test_pred, y))